In [1]:
#%% libraries
import os
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime
%matplotlib inline

# run for jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
def date_stitcher(ybegin, yend,
                  months = ['{num:02d}'.format(num=x) for x in range(1, 13)], 
                  days= ['{num:02d}'.format(num=x) for x in range(1, 32)] ):
    years = [str(x) for x in range(ybegin, yend)]
    filelist = []
    combinations = list(itertools.product(years, months, days))
    for combination in combinations:
        arguments = "_".join(combination)
        command = 'wfreqs_TKP_' + arguments + '.csv'
        filelist.append(command)
    return filelist


def kw_search(flist, keywords, inpdir,fuzzymatch=False,verbose=False):
    freqcols = ['word', 'freq']
    # initialise dataframe
    tallies = pd.DataFrame(flist, columns=['file']).set_index('file')
    tallies['exists']=np.nan
    tallies['wordcount']=np.nan
    for kw in keywords:
        tallies[kw] = np.nan
    # count in all files
    for infile in flist:
        file = inpdir + '/' + infile
        exist_flag = os.path.exists(file)
        if exist_flag:
            tallies.at[infile, 'exists'] = 1
            tkp = pd.read_csv(file, usecols=freqcols, index_col='word')
            tallies.at[infile, 'wordcount'] = tkp.freq.sum()
            for kw in keywords:
                try:
                    # fuzzy match 
                    if fuzzymatch:
                        tkp['word'] = tkp.index
                        tallies.at[infile, kw] = tkp[tkp['word'].str.contains(kw)]['freq'].sum()
                    else:
                    # strict match
                        tallies.at[infile, kw] = pd.to_numeric(tkp.loc[kw])[0]
                    
                except:
                    continue
            if verbose: print(infile, 'loaded and searched')
            del tkp
        else:
            tallies.at[infile, 'exists'] = 0
            if verbose: print(infile, ' does not exist')
            continue
    # subset to nonempty rows
    data = tallies[(tallies['exists']==1)]
    return(data)

def prep_ngram_data(flist,keywords,inp,fuzzymatch=False,percent=False):
    data = kw_search(flist, keywords, inp, fuzzymatch)
    # this will print a barrage of warnings 
    data.reset_index(level=0, inplace=True)
    data.file = data.file.str[11:21] # hacky - relies on particular naming format
    data['date']=pd.to_datetime(data['file'], format='%Y_%m_%d')
    keepvars = ['date','wordcount']+keywords
    
    # normalize by day word-count 
    if percent:
        for k in keywords:
            data[k] = (data[k]/data['wordcount'])*100
        
    clean = data[keepvars].set_index('date')
    
    return(clean)

`prep_ngram_data` optionally applies normalisation on word-counts to construct
$$
P_{w,t} = \frac{N_{w,t}}{\sum_{i}^I N_{i,t}}
$$

for word $w$ on day $t$, where the denominator is the word count 

## interactive plot (uses plotly)

In [3]:
def plot_interactive(df,vars,header='Appearances in TKP archive'):
    data = []
    for v in vars:
        data.append(go.Scatter(x = df.index,y = df[v],name=v))
    
    layout = dict(
        title=header,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,label='1m',step='month',stepmode='backward'),
                    dict(count=6,label='6m',step='month',stepmode='backward'),
                    dict(count=1,label='YTD',step='year',stepmode='todate'),
                    dict(count=1,label='1y',step='year',stepmode='backward'),
                    dict(count=2,label='2y',step='year',stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(),
            type='date'
        )
    )
    fig = dict(data=data, layout=layout)
    return py.iplot(fig)

In [4]:
# preliminaries
working = '/media/alal/NEPALINEWS/The Kathmandu Post'
os.chdir(working)
tmp = '/home/alal/tmp'
inp = '/media/alal/NEPALINEWS/The Kathmandu Post/word_frequencies'

# set up list for date ranges
flist = date_stitcher(2007, 2018)
keywords = ['language', 'bhasa', 'maithili','newar', 'tharu', 'tamang']
langs_nep = ['maithili', 'bhojpuri', 'tharu', 'tamang', 'newar', 'bajjika',
             'magar','dotyali', 'urdu', 'awadhi', 'limbu', 'gurung', 'baitadeli', 'rai', 'aachami']
disasters = ['earthquake', 'fire', 'flood','drought','landslide']

In [23]:
%%time
clean = prep_ngram_data(flist,keywords,inp,fuzzymatch=True,percent=True)

CPU times: user 1min 50s, sys: 1e+03 ms, total: 1min 51s
Wall time: 1min 51s


In [26]:
clean.head()

,wordcount,language,bhasa,maithili,newar,tharu,tamang
date,,,,,,,
2007-03-27,16336.0,0.000122,0.0,0.0,0.000000,0.000000,0.000000
2007-03-28,15718.0,0.000254,0.0,0.0,0.000127,0.000127,0.000127
2007-03-29,16547.0,0.000121,0.0,0.0,0.000060,0.000000,0.000000
2007-03-30,15374.0,0.000260,0.0,0.0,0.000065,0.000000,0.000000
2007-03-31,14247.0,0.000070,0.0,0.0,0.000070,0.000000,0.000070


In [34]:
%%time
langs = prep_ngram_data(flist,langs_nep,inp,fuzzymatch=True)

CPU times: user 4min 14s, sys: 620 ms, total: 4min 15s
Wall time: 4min 16s


In [5]:
%%time
stressors = prep_ngram_data(flist,disasters,inp,fuzzymatch=True)

CPU times: user 1min 13s, sys: 544 ms, total: 1min 14s
Wall time: 1min 14s


# Interactive Plots

check that daily word-counts don't bounce around too much over time - that would mean trends in raw counts are misleading

In [31]:
plot_interactive(clean,['wordcount'],header='word count TKP')

In [32]:
plot_interactive(clean,keywords)

In [35]:
plot_interactive(langs,langs_nep)

The draw time for this plot will be slow for clients without much RAM.


/home/alal/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [6]:
plot_interactive(stressors,disasters)